In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import *
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle

In [2]:
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

In [3]:
def create_browser():
    """
    Function creating browser object.
    """
    # add options to disable pop-ups
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    # return browser object
    chrome = Chrome(options=chrome_options)
    return chrome

In [4]:
def get_user_page_first_time(chrome, user):
    """
    Function loading the basic page.
    """
    # load page
    chrome.get(f"https://www.filmweb.pl/user/{user}/films")
    # agree to terms of use
    chrome.find_element(By.ID, 'didomi-notice-agree-button').click()
    # wait until the advertisement ends
    time.sleep(35)
    chrome.maximize_window()
    time.sleep(10)

In [5]:
def get_user_page(chrome, user):
    """
    Function to get the user page.
    """
    # load page
    chrome.get(f"https://www.filmweb.pl/user/{user}/films")
    time.sleep(10)

In [6]:
def login_to_facebook(chrome, email, user_password):
    """
    Function logging to Facebook.
    """
    for i in range(20):
        chrome.execute_script(f"window.scrollBy(0,{i})","")
        
    chrome.find_element(By.CLASS_NAME, 'facebookLoginButton__button').click()
    
    # get window to scroll that showed up
    scrollable = chrome.find_element(By.CLASS_NAME, '_4t2a')
    chrome.execute_script("return arguments[0].scrollIntoView(true);", scrollable)
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # click whatever
    button = chrome.find_element(By.CLASS_NAME, '_9xo5')
    button.click()
    
    # get all elements needed to log in
    login = chrome.find_element(By.ID, 'email')
    password = chrome.find_element(By.ID, 'pass')
    submit = chrome.find_element(By.ID, 'loginbutton')
    
    # enter credentials
    # login.click()
    login.send_keys(email)
    # password.click()
    password.send_keys(user_password)
    
    # click submit
    submit.click()
    
    time.sleep(10)

In [12]:
def get_genres_and_actors(chrome):
    div = chrome.find_elements(By.CLASS_NAME, 'preview__details')
    all_films = []
    for div_element in div:
        per_film = div_element.find_elements(By.CSS_SELECTOR, 'a')
        one_film_list = []
        for att in per_film:
            res = att.get_attribute('textContent')
            one_film_list.append(res)
        formatted = (tuple(one_film_list[:-2]), tuple(one_film_list[-2:]))
        all_films.append(formatted)
    return all_films

In [7]:
def get_ratings(chrome, user):
    """
    Function returning ratings from the first page.
    """
    
    # scroll down to load ratings
    for i in range(1000):
        chrome.execute_script(f"window.scrollBy(0,{i})","")
    
    # get ratings
    ratings = chrome.find_elements(By.CLASS_NAME, 'userRate__rate')
    ratings_list = [r.get_attribute('textContent') for r in ratings]

    # get film names
    film_names = chrome.find_elements(By.CLASS_NAME, 'preview__link')
    film_names_list = [f.get_attribute('textContent') for f in film_names]
    
    # get year of production
    preview_years = chrome.find_elements(By.CLASS_NAME, 'preview__year')
    preview_years_list = [p.get_attribute('textContent') for p in preview_years]
    
    # get genres and actors
    genres_actors_list = get_genres_and_actors(chrome)
    
    # get community ratings
    community_ratings = chrome.find_elements(By.CLASS_NAME, 'communityRatings__value')
    community_ratings_list = [r.get_attribute('textContent').strip() for r in community_ratings]
    community_ratings_list = community_ratings_list[::2]
    
    # number of ratings
    community_ratings_number = chrome.find_elements(By.CLASS_NAME, 'communityRatings__description')
    ratings_number_list = [r.get_attribute('textContent').strip() for r in community_ratings_number]
    ratings_number_list = [i[:i.find('ocen')].replace(' ', '') for i in ratings_number_list]
    ratings_number_list = list(filter(('chcezobaczy').__ne__, ratings_number_list))
    
    # add user name
    user_name_list = len(ratings) * [user]
    
    # list of tuples
    zipped_values = list(zip(user_name_list,
                             film_names_list, 
                             preview_years_list,
                             genres_actors_list,
                             community_ratings_list,
                             ratings_number_list,
                             ratings_list))
    
    
    
    return zipped_values
    

In [13]:
def get_all_user_ratings(chrome, user):
    """
    Function returning all ratings of given user.
    The results are written to pickle files.
    """
    # get ratings from the first page
    first = get_ratings(chrome, user)
    print(first)
    
    # prepare placeholders for ratings
    old = []
    new = first
    
    # starting page number
    page = 2
    
    # iterate as long as there are films ratings on the next pages
    while (len(new) > 0) and (old != new):
        print(page)
        chrome.get(f"https://www.filmweb.pl/user/{user}/films?page={page}")
        x = get_ratings(chrome, user)
        print(x)
        page += 1
        old, new = old + new, x
        
    if len(old) > 0:
        with open(f"{user}_full_data.pickle", 'wb') as handle:
            pickle.dump(old, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
def get_user_friends(chrome, user):
    """
    Function returning the list of user friends.
    """
    chrome.get(f"https://www.filmweb.pl/user/{user}/friends")
    friends = chrome.find_elements(By.CLASS_NAME, 'user__nick')
    friends_list = [f.get_attribute('textContent').strip() for f in friends]
    
    return friends_list

In [11]:
def get_ratings_for_all_friends(chrome, user):
    """
    Summary function: get all ratings for all friends.
    """
    friends_list = get_user_friends(chrome, user)
    for user in friends_list:
        print(f"Displaying results for user {user}")
        chrome.get(f"https://www.filmweb.pl/user/{user}/films")
        get_all_user_ratings(chrome, user)

# SCRAPPING YOURSELF AND YOUR FRIENDS

In [14]:
# EXAMPLE USE

# 1. create browser object.
chrome = create_browser()

# 2. load the first page, accept usage terms etc.
get_user_page_first_time(chrome, "honorciak")

# 3. login to Facebook. In case it fails, just do it by hand in this special browser window.
login_to_facebook(chrome, "<email>", "<password>")

# 4. congrats! you are ready to get the records
get_ratings_for_all_friends(chrome, "honorciak")

# 5. if something fails in between, you can get records just for one user
#get_all_user_ratings(chrome, user)

Displaying results for user honorciak
[('honorciak', 'Skazani na Shawshank', '1994', (('Dramat',), ('Tim Robbins', 'Morgan Freeman')), '8,8', '907894', '9'), ('honorciak', 'Zielona mila', '1999', (('Dramat',), ('Tom Hanks', 'David Morse')), '8,6', '988839', '10'), ('honorciak', 'Anna', '2019', (('Thriller', 'Akcja'), ('Sasha Luss', 'Helen Mirren')), '6,5', '24770', '6'), ('honorciak', 'Gwiezdne wojny: Część III - Zemsta Sithów', '2005', (('Przygodowy', 'Sci-Fi'), ('Ewan McGregor', 'Natalie Portman')), '7,6', '252090', '3'), ('honorciak', 'Cztery wesela i pogrzeb', '1994', (('Komedia rom.',), ('Hugh Grant', 'Andie MacDowell')), '6,9', '96711', '7'), ('honorciak', 'Masz wiadomość', '1998', (('Komedia rom.',), ('Tom Hanks', 'Meg Ryan')), '6,8', '158125', '8'), ('honorciak', 'Aż do kości', '2017', (('Dramat',), ('Lily Collins', 'Keanu Reeves')), '6,9', '75960', '8'), ('honorciak', 'Holiday', '2006', (('Komedia rom.', 'Świąteczny'), ('Cameron Diaz', 'Kate Winslet')), '7,2', '196795', '7'), 

[('honorciak', 'Shrek', '2001', (('Animacja', 'Familijny', 'Komedia'), ('Mike Myers', 'Eddie Murphy')), '7,8', '825315', '9'), ('honorciak', 'Przekręt', '2000', (('Komedia kryminalna',), ('Jason Statham', 'Stephen Graham')), '8,0', '252520', '8'), ('honorciak', 'Bohemian Rhapsody', '2018', (('Biograficzny', 'Dramat', 'Muzyczny'), ('Rami Malek', 'Lucy Boynton')), '7,9', '214523', '8'), ('honorciak', 'Pamiętnik', '2004', (('Melodramat',), ('Ryan Gosling', 'Rachel McAdams')), '8,0', '368736', '7'), ('honorciak', 'Buntownik z wyboru', '1997', (('Dramat obyczajowy',), ('Matt Damon', 'Robin Williams')), '8,1', '281720', '9'), ('honorciak', 'Whiplash', '2014', (('Dramat', 'Muzyczny'), ('Miles Teller', 'J.K. Simmons')), '8,1', '295730', '8'), ('honorciak', 'Chłopiec w pasiastej piżamie', '2008', (('Dramat', 'Wojenny'), ('Asa Butterfield', 'Jack Scanlon')), '8,1', '401406', '8'), ('honorciak', 'Piękny umysł', '2001', (('Biograficzny', 'Dramat'), ('Russell Crowe', 'Ed Harris')), '8,3', '499120',

[('mat1998mat', 'Parasite', '2019', (('Dramat',), ('Kang-ho Song', 'Seon-gyun Lee')), '8,1', '164632', '9'), ('mat1998mat', 'Obiecująca. Młoda. Kobieta.', '2020', (('Dramat', 'Kryminał'), ('Carey Mulligan', 'Bo Burnham')), '7,6', '52743', '9'), ('mat1998mat', 'Borat: Podpatrzone w Ameryce, aby Kazachstan rósł w siłę, a ludzie żyli dostatniej', '2006', (('Komedia',), ('Sacha Baron Cohen', 'Ken Davitian')), '6,6', '189303', '7')]
5
[]
Displaying results for user zuzannagrzadziel
[]
Displaying results for user Michael1997c
[('Michael1997c', 'Logan: Wolverine', '2017', (('Dramat', 'Akcja', 'Sci-Fi'), ('Hugh Jackman', 'Patrick Stewart')), '7,6', '166425', '8'), ('Michael1997c', 'Kapitan Ameryka: Wojna bohaterów', '2016', (('Akcja', 'Sci-Fi'), ('Chris Evans', 'Robert Downey Jr.')), '7,5', '178133', '9'), ('Michael1997c', 'Avengers: Czas Ultrona', '2015', (('Akcja', 'Sci-Fi'), ('Robert Downey Jr.', 'Chris Hemsworth')), '7,2', '203946', '6'), ('Michael1997c', 'Ant-Man', '2015', (('Akcja', 'Sci

[('kajti98', 'Nie czas umierać', '2021', (('Sensacyjny',), ('Daniel Craig', 'Léa Seydoux')), '6,9', '76435', '8'), ('kajti98', 'Lighthouse', '2019', (('Dramat', 'Horror'), ('Willem Dafoe', 'Robert Pattinson')), '7,5', '59543', '8'), ('kajti98', 'Jojo Rabbit', '2019', (('Dramat', 'Komedia', 'Wojenny'), ('Roman Griffin Davis', 'Thomasin McKenzie')), '7,7', '104206', '9'), ('kajti98', 'Sok z żuka', '1988', (('Fantasy', 'Czarna komedia'), ('Alec Baldwin', 'Geena Davis')), '7,2', '175524', '9'), ('kajti98', 'Coco', '2017', (('Animacja', 'Familijny', 'Przygodowy'), ('Anthony Gonzalez', 'Gael García Bernal')), '8,3', '157426', '8'), ('kajti98', 'Niefortunny numerek lub szalone porno', '2021', (('Dramat',), ('Katia Pascariu', 'Claudia Ieremia')), '6,6', '4875', '7'), ('kajti98', 'Erotica 2022', '2020', (('Dramat', 'Sci-Fi'), ('Małgorzata Bela', 'Agnieszka Żulewska')), '2,8', '8523', '2'), ('kajti98', 'Baby Bump', '2015', (('Dramat', 'Surrealistyczny'), ('Kacper Olszewski', 'Agnieszka Podsiadli

[('kajti98', 'Dom, który zbudował Jack', '2018', (('Dramat',), ('Matt Dillon', 'Bruno Ganz')), '6,6', '25427', '6'), ('kajti98', 'Twój Vincent', '2017', (('Animacja', 'Dramat', 'Kryminał'), ('Douglas Booth', 'Jerome Flynn')), '7,9', '79436', '8'), ('kajti98', 'Cyrk motyli', '2009', (('Dramat', 'Krótkometrażowy'), ('Eduardo Verástegui', 'Nick Vujicic')), '7,8', '52785', '7'), ('kajti98', 'Mucha', '1986', (('Horror', 'Sci-Fi'), ('Jeff Goldblum', 'Geena Davis')), '7,1', '40364', '7'), ('kajti98', 'Mucha', '1958', (('Horror', 'Sci-Fi'), ('David Hedison', 'Patricia Owens')), '6,8', '2040', '8'), ('kajti98', 'Medicus', '2013', (('Przygodowy', 'Dramat historyczny'), ('Tom Payne', 'Ben Kingsley')), '7,4', '33552', '6'), ('kajti98', 'Riff-Raff', '1991', (('Komedia', 'Dramat obyczajowy'), ('Emer McCourt', 'George Moss')), '7,0', '452', '7'), ('kajti98', 'Imperium zmysłów', '1976', (('Erotyczny', 'Melodramat'), ('Tatsuya Fuji', 'Eiko Matsuda')), '5,7', '4610', '2'), ('kajti98', 'Joker', '2019', (

...

[('Marlena_96', 'Pachnidło: Historia mordercy', '2006', (('Dramat', 'Thriller'), ('Ben Whishaw', 'Alan Rickman')), '7,4', '320444', '9'), ('Marlena_96', 'World War Z', '2013', (('Horror', 'Akcja'), ('Brad Pitt', 'Mireille Enos')), '6,6', '188543', '8'), ('Marlena_96', 'Nimfomanka - Część I', '2013', (('Dramat', 'Erotyczny'), ('Charlotte Gainsbourg', 'Stellan Skarsgård')), '6,0', '72825', '7'), ('Marlena_96', 'Wiecznie żywy', '2013', (('Horror', 'Komedia'), ('Nicholas Hoult', 'Teresa Palmer')), '6,4', '71272', '7'), ('Marlena_96', 'Honey', '2003', (('Melodramat', 'Muzyczny'), ('Jessica Alba', 'Romeo Miller')), '6,7', '55933', '7'), ('Marlena_96', '17 Again', '2009', (('Komedia', 'Dla młodzieży'), ('Zac Efron', 'Matthew Perry')), '6,2', '104053', '7'), ('Marlena_96', 'Listy do Julii', '2010', (('Dramat', 'Komedia', 'Romans'), ('Christopher Egan', 'Amanda Seyfried')), '6,8', '107219', '9'), ('Marlena_96', 'Łatwa dziewczyna', '2010', (('Komedia',), ('Emma Stone', 'Penn Badgley')), '6,5', '

[('Marlena_96', 'Nostalgia anioła', '2009', (('Dramat', 'Fantasy'), ('Saoirse Ronan', 'Mark Wahlberg')), '7,1', '122326', '9'), ('Marlena_96', 'Big Love', '2012', (('Melodramat', 'Thriller'), ('Aleksandra Hamkało', 'Antoni Pawlicki')), '5,8', '74311', '4'), ('Marlena_96', 'Zakochana bez pamięci', '2012', (('Dramat', 'Komedia', 'Romans'), ('Juliette Binoche', 'Mathieu Kassovitz')), '6,7', '8895', '7'), ('Marlena_96', 'Czas na miłość', '2013', (('Dramat', 'Fantasy', 'Komedia', 'Romans'), ('Domhnall Gleeson', 'Rachel McAdams')), '7,6', '195906', '8'), ('Marlena_96', 'Droga do szczęścia', '2008', (('Melodramat',), ('Kate Winslet', 'Leonardo DiCaprio')), '7,2', '127197', '8'), ('Marlena_96', 'Igrzyska śmierci: W pierścieniu ognia', '2013', (('Akcja', 'Sci-Fi'), ('Jennifer Lawrence', 'Josh Hutcherson')), '7,3', '285086', '10'), ('Marlena_96', 'Liceum Avalon', '2010', (('Dramat', 'Familijny', 'Fantasy'), ('Britt Robertson', 'Gregg Sulkin')), '5,7', '14322', '5'), ('Marlena_96', 'Złodziejka ks

[('Marlena_96', 'Radio', '2003', (('Dramat obyczajowy', 'Sportowy'), ('Cuba Gooding Jr.', 'Ed Harris')), '7,8', '34088', '10'), ('Marlena_96', 'Hobbit: Niezwykła podróż', '2012', (('Fantasy', 'Przygodowy'), ('Martin Freeman', 'Ian McKellen')), '7,6', '393140', '10'), ('Marlena_96', 'Hobbit: Pustkowie Smauga', '2013', (('Fantasy', 'Przygodowy'), ('Martin Freeman', 'Ian McKellen')), '7,6', '317640', '10'), ('Marlena_96', 'Szeregowiec Ryan', '1998', (('Dramat', 'Wojenny'), ('Tom Hanks', 'Tom Sizemore')), '8,1', '644848', '10'), ('Marlena_96', 'Zielona mila', '1999', (('Dramat',), ('Tom Hanks', 'David Morse')), '8,6', '988839', '10'), ('Marlena_96', 'Władca Pierścieni: Drużyna Pierścienia', '2001', (('Fantasy', 'Przygodowy'), ('Elijah Wood', 'Sean Astin')), '8,0', '718502', '10'), ('Marlena_96', 'Cast Away - poza światem', '2000', (('Dramat',), ('Tom Hanks', 'Helen Hunt')), '7,5', '579724', '9'), ('Marlena_96', 'Shrek', '2001', (('Animacja', 'Familijny', 'Komedia'), ('Mike Myers', 'Eddie M

[('piotrr99', 'Big Lebowski', '1998', (('Komedia kryminalna',), ('Jeff Bridges', 'John Goodman')), '7,4', '150866', '6'), ('piotrr99', 'Jeden gniewny człowiek', '2021', (('Thriller', 'Akcja'), ('Jason Statham', 'Holt McCallany')), '7,0', '27582', '5'), ('piotrr99', 'Porachunki', '1998', (('Komedia kryminalna',), ('Jason Flemyng', 'Nick Moran')), '7,9', '107869', '8'), ('piotrr99', 'Pieskie popołudnie', '1975', (('Dramat', 'Thriller'), ('Al Pacino', 'John Cazale')), '7,7', '44716', '8'), ('piotrr99', 'Blow', '2001', (('Biograficzny', 'Dramat', 'Kryminał'), ('Johnny Depp', 'Penélope Cruz')), '7,8', '223363', '6'), ('piotrr99', 'Carne', '1991', (('Dramat', 'Thriller', 'Krótkometrażowy'), ('Philippe Nahon', 'Lucile Hadzihalilovic')), '6,7', '1850', '7'), ('piotrr99', 'Dym', '1995', (('Dramat', 'Komedia'), ('Giancarlo Esposito', 'Harvey Keitel')), '7,7', '20517', '8'), ('piotrr99', 'Ojciec', '2020', (('Dramat',), ('Anthony Hopkins', 'Olivia Colman')), '8,2', '41155', '9'), ('piotrr99', 'Nom

[('piotrr99', 'Władca Pierścieni: Powrót króla', '2003', (('Fantasy', 'Przygodowy'), ('Elijah Wood', 'Sean Astin')), '8,4', '518843', '10'), ('piotrr99', 'Perfekcja', '2018', (('Thriller',), ('Alaina Huffman', 'Allison Williams')), '5,5', '9574', '10'), ('piotrr99', 'Koneser', '2013', (('Dramat',), ('Geoffrey Rush', 'Jim Sturgess')), '7,9', '150276', '8'), ('piotrr99', 'Zawrót głowy', '1958', (('Melodramat', 'Thriller', 'Psychologiczny'), ('James Stewart', 'Kim Novak')), '7,7', '36738', '7'), ('piotrr99', 'Wszystko za życie', '2007', (('Dramat', 'Biograficzny', 'Przygodowy'), ('Emile Hirsch', 'Vince Vaughn')), '7,9', '261757', '7'), ('piotrr99', 'Green Book', '2018', (('Dramat', 'Komedia'), ('Viggo Mortensen', 'Mahershala Ali')), '8,2', '214152', '7'), ('piotrr99', 'Bar', '2017', (('Horror', 'Komedia'), ('Blanca Suárez', 'Mario Casas')), '5,8', '6454', '8'), ('piotrr99', 'Mechaniczna pomarańcza', '1971', (('Dramat', 'Sci-Fi'), ('Malcolm McDowell', 'Patrick Magee')), '7,9', '214189', '7

# SCRAPPING TOP 500 USERS

In [1]:
def get_most_active_users_one_page(chrome, page):
    if page == 1:
        chrome.get('https://www.filmweb.pl/users/ranking/usersMonthly')
    else:
        chrome.get(f"https://www.filmweb.pl/users/ranking/usersMonthly?page={page}")
    friends = chrome.find_elements(By.CLASS_NAME, 'rankingList__userName')
    friends_list = [f.get_attribute('textContent').strip() for f in friends]
    return friends_list

In [2]:
def get_500_most_active_users(chrome):
    users_list = []
    for page_num in range(10):
        users_per_page = get_most_active_users_one_page(chrome, page_num + 1)
        users_list = users_list + users_per_page
    return users_list

In [ ]:
active_500_users = get_500_most_active_users(chrome)

for user in active_500_users:
    get_user_page(chrome, user)
    print(f"Displaying results for user {user}")
    ratings = get_ratings(chrome, user)
    with open(f"{user}_full_data.pickle", 'wb') as handle:
        pickle.dump(ratings, handle, protocol=pickle.HIGHEST_PROTOCOL)